# CHAT BOT USING DEEP LEARNING AND NLP

In [22]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [23]:
# Loading the intents json file that we had created

In [24]:
with open('intents.json') as file:
    data = json.load(file)

In [25]:
# Now we creat empty lists and write a nested loop to grab and store the patterns and responses into it

In [26]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag']  not in labels:
        labels.append(intent['tag'])

In [27]:
num_classes = len(labels)
num_classes

8

In [28]:
labels

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint']

In [ ]:
# Here we label encode the training_labels

In [30]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [29]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'complaint',
 'complaint',
 'complaint']

In [31]:
# Now we label encode the training labels
le = LabelEncoder()
le.fit(training_labels)
training_labels = le.transform(training_labels)

In [32]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1], dtype=int64)

# Tokening the data

In [33]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"


In [34]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)


In [35]:
tokenizer.fit_on_texts(training_sentences)

In [36]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [37]:
word_index = tokenizer.word_index

In [38]:
word_index

{'<OOV>': 1,
 'you': 2,
 'a': 3,
 'i': 4,
 'me': 5,
 'to': 6,
 'account': 7,
 'help': 8,
 'what': 9,
 'is': 10,
 'for': 11,
 'are': 12,
 'can': 13,
 'need': 14,
 'create': 15,
 'new': 16,
 'complaint': 17,
 'there': 18,
 'thanks': 19,
 'who': 20,
 'your': 21,
 'name': 22,
 'please': 23,
 'support': 24,
 'how': 25,
 'open': 26,
 'want': 27,
 'an': 28,
 'hi': 29,
 'hey': 30,
 'anyone': 31,
 'hello': 32,
 'hay': 33,
 'bye': 34,
 'see': 35,
 'later': 36,
 'goodbye': 37,
 'thank': 38,
 "that's": 39,
 'helpful': 40,
 'the': 41,
 'should': 42,
 'call': 43,
 'whats': 44,
 'could': 45,
 'give': 46,
 'hand': 47,
 'do': 48,
 'have': 49,
 'raise': 50,
 'about': 51,
 'service': 52}

In [39]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [40]:
sequences

[[29],
 [30],
 [10, 31, 18],
 [32],
 [33],
 [34],
 [35, 2, 36],
 [37],
 [19],
 [38, 2],
 [39, 40],
 [19, 11, 41, 8],
 [20, 12, 2],
 [9, 12, 2],
 [20, 2, 12],
 [9, 10, 21, 22],
 [9, 42, 4, 43, 2],
 [44, 21, 22],
 [45, 2, 8, 5],
 [46, 5, 3, 47, 23],
 [13, 2, 8],
 [9, 13, 2, 48, 11, 5],
 [4, 14, 3, 24],
 [4, 14, 3, 8],
 [24, 5, 23],
 [4, 14, 6, 15, 3, 16, 7],
 [25, 6, 26, 3, 16, 7],
 [4, 27, 6, 15, 28, 7],
 [13, 2, 15, 28, 7, 11, 5],
 [25, 6, 26, 3, 16, 7],
 [49, 3, 17],
 [4, 27, 6, 50, 3, 17],
 [18, 10, 3, 17, 51, 3, 52]]

In [41]:
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [42]:
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 29],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 10, 31, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 32],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 33],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 34],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 35,  2, 36],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 37],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 38,  2],
       [ 0,  0,  0,  0,  0,  0

# Creating a model using deep learning

In [45]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [46]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 0s/step - loss: 2.0800 - accuracy: 0.0919
Epoch 2/500
2/2 [==============================] - 0s 0s/step - loss: 2.0782 - accuracy: 0.2449
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0771 - accuracy: 0.2143
Epoch 4/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0768 - accuracy: 0.2039
Epoch 5/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0758 - accuracy: 0.2039
Epoch 6/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0745 - accuracy: 0.2143
Epoch 7/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0734 - accuracy: 0.2143
Epoch 8/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0724 - accuracy: 0.2143
Epoch 9/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0720 - accuracy: 0.2039
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0708 - accuracy: 0.2143
Epoch 11/500
2/2 [============

2/2 [==============================] - 0s 3ms/step - loss: 1.8244 - accuracy: 0.3368
Epoch 167/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8232 - accuracy: 0.3264
Epoch 168/500
2/2 [==============================] - 0s 4ms/step - loss: 1.8167 - accuracy: 0.3368
Epoch 169/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8121 - accuracy: 0.3368
Epoch 170/500
2/2 [==============================] - 0s 4ms/step - loss: 1.8070 - accuracy: 0.3368
Epoch 171/500
2/2 [==============================] - 0s 4ms/step - loss: 1.8073 - accuracy: 0.3368
Epoch 172/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8017 - accuracy: 0.3368
Epoch 173/500
2/2 [==============================] - 0s 5ms/step - loss: 1.7959 - accuracy: 0.3368
Epoch 174/500
2/2 [==============================] - 0s 4ms/step - loss: 1.7971 - accuracy: 0.3368
Epoch 175/500
2/2 [==============================] - 0s 4ms/step - loss: 1.7956 - accuracy: 0.3980
Epoch 176/500
2/2 [=====

2/2 [==============================] - 0s 3ms/step - loss: 1.0423 - accuracy: 0.6124
Epoch 331/500
2/2 [==============================] - 0s 4ms/step - loss: 1.0566 - accuracy: 0.6020
Epoch 332/500
2/2 [==============================] - 0s 4ms/step - loss: 1.0449 - accuracy: 0.6020
Epoch 333/500
2/2 [==============================] - 0s 6ms/step - loss: 1.0439 - accuracy: 0.6020
Epoch 334/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0273 - accuracy: 0.6632
Epoch 335/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0280 - accuracy: 0.6632
Epoch 336/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0132 - accuracy: 0.6430
Epoch 337/500
2/2 [==============================] - 0s 4ms/step - loss: 1.0131 - accuracy: 0.6326
Epoch 338/500
2/2 [==============================] - 0s 4ms/step - loss: 1.0038 - accuracy: 0.6124
Epoch 339/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0066 - accuracy: 0.6326
Epoch 340/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.5659 - accuracy: 0.8469
Epoch 495/500
2/2 [==============================] - 0s 6ms/step - loss: 0.5514 - accuracy: 0.8469
Epoch 496/500
2/2 [==============================] - 0s 4ms/step - loss: 0.5622 - accuracy: 0.8469
Epoch 497/500
2/2 [==============================] - 0s 4ms/step - loss: 0.5578 - accuracy: 0.8469
Epoch 498/500
2/2 [==============================] - 0s 4ms/step - loss: 0.5536 - accuracy: 0.8469
Epoch 499/500
2/2 [==============================] - 0s 3ms/step - loss: 0.5484 - accuracy: 0.8469
Epoch 500/500
2/2 [==============================] - 0s 5ms/step - loss: 0.5453 - accuracy: 0.8469


In [48]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(le, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chat_model\assets


# Running the chat model

In [ ]:
import colorama 
colorama.init()
from colorama import Fore, Style, Back


In [ ]:
# Creating an interective framerwork for chatting 

In [ ]:
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
ChatBot: Hello
User: how are you 
ChatBot: Hi there
User: register
ChatBot: Hi
User: register a complaint
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: 